# Vehicle Type Classification

In [12]:
import pandas as pd

# Load the dataset
file_path = "/Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RoadTrafficNoise/IDMT-Traffic/datasets/df_main_encoded_only.csv"

# Load the dataset
df_vehicle_classification = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

# Check column names and data types
print(df.info())

   Unnamed: 0                                               file  \
0          18  2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1116695...   
1          19  2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1116695...   
2          20  2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1137602...   
3          21  2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1137602...   
4          24  2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_1141442...   

   is_background_encoded  date_time_encoded  location_encoded  \
0                      0                  0                 0   
1                      0                  0                 0   
2                      0                  0                 0   
3                      0                  0                 0   
4                      0                  0                 0   

   speed_kmh_encoded  daytime_encoded  weather_encoded  vehicle_encoded  \
0                  0                1                0                1   
1                  0                1             

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# Extract features and target
X = df_vehicle_classification.drop(columns=['file', 'vehicle_encoded'])  # Drop "file" and "vehicle_encoded"
y = df_vehicle_classification['vehicle_encoded']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize and train the XGBoost classifier
xgb_clf = XGBClassifier(eval_metric='mlogloss', random_state=42)
xgb_clf.fit(X_train, y_train)

# Make predictions
y_pred = xgb_clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.67      0.74        21
           1       0.92      0.98      0.95      1561
           2       1.00      0.99      0.99        86
           3       0.71      0.36      0.47       205

    accuracy                           0.91      1873
   macro avg       0.86      0.75      0.79      1873
weighted avg       0.90      0.91      0.90      1873



Key Observations:

    Class 1 Dominance:
        Class 1 has the highest support (1561 samples), and the model performs very well on it with a high F1-score (0.95).
        This may indicate that the model is biased towards this class due to its prevalence in the dataset.

    Poor Performance on Class 3:
        Class 3 has a relatively lower recall (0.36) and F1-score (0.47), indicating that the model struggles to correctly identify samples of this class.
        This could be due to insufficient data for this class or features that do not distinguish it well from others.

    Macro Average vs. Weighted Average:
        The macro average shows the model's performance across all classes equally (unweighted), which is lower (F1-score: 0.79) due to poor performance on minority classes.
        The weighted average is higher (F1-score: 0.90), influenced by the dominance of Class 1.

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from xgboost import XGBClassifier, DMatrix

# Extract features and target
X = df[features]
y = df['vehicle']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the pipeline
pipeline = Pipeline([
    ('xgb', XGBClassifier(eval_metric='mlogloss'))  # Multiclass support
])

# Define a smaller parameter grid for GridSearchCV
param_grid = {
    'xgb__n_estimators': [100],             # Number of boosting rounds
    'xgb__learning_rate': [0.1, 0.3],       # Learning rate (eta)
    'xgb__max_depth': [3],                  # Maximum depth of trees
    'xgb__gamma': [0, 1],                   # Minimum loss reduction to split
    'xgb__reg_alpha': [0, 0.1],             # L1 regularization term
    'xgb__reg_lambda': [1, 1.5]             # L2 regularization term
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,                 # 5-fold cross-validation
    scoring='f1_macro',   # Optimize for macro F1 score for multiclass
    verbose=1,
    n_jobs=-1             # Use all available CPUs
)

# Perform Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_mapping.values()))

# Save the label mapping
print("Label Mapping (Numeric to Vehicle):")
print(label_mapping)

NameError: name 'features' is not defined